In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence

import numpy as np
from vgg16 import Vgg16

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import PIL.Image

import json
from tqdm import tqdm

from keras.optimizers import SGD, RMSprop, Adam

from utils import *
import cPickle as pickle
from matplotlib import pyplot as plt

import collections

import preprocessing as preproc

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
save_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/"

train_path = save_path + "train/"
val_path = save_path + "val/"

captions_folder = "captions/"
general_datastruct_folder = "general-datastruct/"

In [3]:
def first_N_captions_sum(captions,N):
    caption_lengths = [len(caption.split()) for caption in captions]
    counter=collections.Counter(caption_lengths)
    return sum([value for key,value in counter.iteritems() if key <= N])

def get_unique_words(captions):
    unique_words = []
    words = [caption.split() for caption in captions]
   
    for word in words:
        unique_words.extend(word)
        
    unique_words = list(set(unique_words))
    
    return unique_words

In [4]:
train_captions = preproc.get_captions_from_batch(train_path + captions_folder, batch_nr = 0)
val_captions = preproc.get_captions_from_batch(val_path + captions_folder, batch_nr = 0)

captions = train_captions + val_captions
len(captions)

IOError: [Errno 2] No such file or directory: '/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/train/captions/captions_batch_0.p'

In [ ]:
totalSum = first_N_captions_sum(captions,500)

N = 9
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ----> Sum = %d  => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 10
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 12
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 15
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 20
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

N = 50
firstN = first_N_captions_sum(captions,N)
print("Captions of length <= %d  ---> Sum = %d => Percentage = %f"%(N,firstN,firstN/float(totalSum)))

In [ ]:
def no_words_with_min_appearances(captions,min_no_of_occurences):
    
    sentences = [caption.split() for caption in captions]
    words = []
    for word in sentences:
        words.extend(word)

    counter=collections.Counter(words)
    nr_words = len([_ for word,no_appearances in counter.most_common() if no_appearances >= min_no_of_occurences])
    
    return nr_words

In [ ]:
unique_words = preproc.load_obj(save_path+general_datastruct_folder+"unique_words")
allWords = float(len(unique_words))
allWords

In [ ]:
N = 1
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 2
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 3
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 5
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 10
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 100
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 500
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 5000
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

# Captions containing only common words

In [ ]:
total_length = len(train_mask_no_words)

MIN_NO_OF_APP = 2
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 3
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 4
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 10
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 50
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))
